In [1]:
!pip install transformers[torch] -q
!pip install seqeval -q
!pip install datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 926.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00


In [2]:
import os
import json
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datasets import load_metric
from typing import List, Dict, Tuple

import torch
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset


from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader
from transformers import (
    BertTokenizerFast,
    BertForTokenClassification,
    DataCollatorForTokenClassification,
    pipeline,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    AutoModel
)
from seqeval.metrics import f1_score

from tqdm import tqdm
tqdm.pandas()

pd.set_option('max_colwidth', None)

## Скачивание данных

In [3]:
aspects_path = '/content/drive/MyDrive/NLP Course/Project/train_aspects.txt'
categories_path = '/content/drive/MyDrive/NLP Course/Project/train_cats.txt'
reviews_path = '/content/drive/MyDrive/NLP Course/Project/train_reviews.txt'

In [4]:
aspects = pd.read_csv(aspects_path, sep='\t', names=['idx', 'category', 'aspect', 'start', 'end', 'mood'])
categories = pd.read_csv(categories_path, sep='\t', names=['idx', 'category', 'mood'])
reviews = pd.read_csv(reviews_path, sep='\t', names=['idx', 'text'])

In [5]:
# aspects['category'].hist()

In [6]:
# all_cats = aspects['category'].unique()
# for cat in all_cats:
#     categories[categories.category==cat]['mood'].hist()
#     plt.title(cat)
#     plt.show()

In [7]:
# model_name = 'deepvk/bert-base-uncased'
# tokenizer = BertTokenizerFast.from_pretrained(model_name)

# reviews['tokens_len'] = reviews.text.progress_apply(lambda text: len(tokenizer.tokenize(text)))

In [8]:
# # все ревью до 300 токенов, что подходит, так как токенайзер принимает на вход до 512 токенов
# reviews.tokens_len.hist()

In [9]:
# aspects.aspect.value_counts()[:20]

In [6]:
train_datapath = '/content/drive/MyDrive/NLP Course/Data/BIO_id_token_tag.csv'
train_data = pd.read_csv(train_datapath, sep='\t')

test_datapath = '/content/drive/MyDrive/NLP Course/Data/test/TEST_BIO.csv'
test_data = pd.read_csv(test_datapath, sep='\t')

In [7]:
train_data.head(2)

,id,bio_token,bio_tag
0,31963,При,O
1,31963,входе,O


In [8]:
train_clean_tokens = train_data.groupby('id')['bio_token'].agg(list)
train_clean_labels = train_data.groupby('id')['bio_tag'].agg(list)

In [9]:
test_clean_tokens = test_data.groupby('id')['bio_token'].agg(list)
test_clean_labels = test_data.groupby('id')['bio_tag'].agg(list)

In [10]:
test_clean_tokens.iloc[[11]]

id
5974    [Отличный, интерьер, !, Приятная, атмосфера, и, живая, музыка, (, аккордеон, ), ., Интересное, оформление, блюд, !, -, Это, все, поднимает, настроение, !, Ожидание, МЕНЮ, более, 15, минут, (, ресторан, был, пустой, ,, праздновали, в, одной, комнате, ,, а, 3, столика, в, главном, зале, ждали, пока, те, закажут, ,, чтоб, нам, передать, меню, !, ), ., Очень, медленное, обслуживание, (, или, мало, поваров, или, официантов, ), ., Ждал, квас, больше, часа, ,, пришлось, пить, воду, на, столе, (, оказалось, за, 300, руб., бутылка, ), ,, потом, принесли, квас, (, в, меню, цена, указана, :, 60, руб, ., за, 0.5, мл, !, ...]
Name: bio_token, dtype: object

In [11]:
test_clean_labels.iloc[[11]]

id
5974    [O, B-Interior, O, O, B-Interior, O, B-Interior, I-Interior, O, O, O, O, O, B-Food, I-Food, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Service, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Service, O, O, O, B-Service, O, B-Service, O, O, B-Service, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...]
Name: bio_tag, dtype: object

## Обучение BERT

### Подготовка данных

In [13]:
model_name = 'DeepPavlov/rubert-base-cased'  # 'deepvk/bert-base-uncased'
tokenizer = BertTokenizerFast.from_pretrained(model_name, model_max_length=512)

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

In [14]:
if 'uncased' in model_name:
    train_clean_tokens = train_clean_tokens.apply(lambda text: [str.lower(word) for word in text])
    test_clean_tokens = test_clean_tokens.apply(lambda text: [str.lower(word) for word in text])

In [15]:
def get_tokens_pairs(texts, labels, if_bio=True):
    '''
    Функция делит слова на токены, подходящие модели,
    и изменяет набор тэгов, чтобы он соответствовал набору токенов
    '''

    res_tokens = []
    res_labels = []
    for text, tags in zip(texts, labels):
        result = [('[CLS]', 'O')]
        all_tokens = ['[CLS]']
        all_labels = ['O']

        for (word, label) in zip(text, tags):
            tokens = tokenizer.tokenize(word)

            all_tokens.extend(tokens)

            if if_bio:
                if label.startswith('B'):
                    all_labels.append(label)
                    all_labels.extend([label.replace('B', 'I')]*(len(tokens)-1))
                else:
                    all_labels.extend([label]*len(tokens))
            else:
                if label == 'O':
                    all_labels.extend([label]*len(tokens))
                else:
                    all_labels.extend([label[2:]]*len(tokens))

        all_tokens.append('[SEP]')
        all_labels.append('O')

        res_tokens.append(all_tokens)
        res_labels.append(all_labels)

    return res_tokens, res_labels

In [16]:
train_tokens, train_labels = get_tokens_pairs(train_clean_tokens, train_clean_labels)
test_tokens, test_labels = get_tokens_pairs(test_clean_tokens, test_clean_labels)

Можно попробовать сделать искусственные аугментации, примерный код ниже, но в таком формате только по категории price не помогает в обучении.

In [38]:
def get_price_augs(num):
    tokens = []
    labels = []

    for _ in range(num):

        if random.choice([False, False, True]):
            value = random.randint(1, 10_000)
            add_value_type = random.choice(['тысяч', 'тыс', 'т'])
            tokens.append(['депозит', str(value), add_value_type])
            labels.append(['B-Price' if i==0 else 'I-Price' for i in range(len(tokens[-1]))])

        else:
            value = random.randint(500, 10_000)
            add_text = random.choice(['с человека', 'на человека', 'на всех', ''])
            add_value_type = random.choice(['руб', 'р', 'рубля', 'р.'])

            if random.choice([False, False, True]):
                value_2 = random.randint(value, 10_000)
                tokens.append([f'{value}-{value_2}', add_value_type, add_text])
                labels.append(['B-Price' if i==0 else 'I-Price' for i in range(len(tokens[-1]))])

            else:
                tokens.append([str(value), add_value_type, add_text])
                labels.append(['B-Price' if i==0 else 'I-Price' for i in range(len(tokens[-1]))])

    return tokens, labels

aug_tokens, aug_labels = get_price_augs(10)

In [39]:
aug_train_tokens, aug_train_labels = get_tokens_pairs(aug_tokens, aug_labels)

In [18]:
train_tokens += aug_train_tokens
train_labels += aug_train_labels

In [17]:
labels_set = set([l for label in train_labels for l in label])

In [18]:
label2id = {
    'B-Food': 0, 'B-Interior': 1, 'B-Price': 2, 'B-Service': 3,
    'B-Whole': 4, 'I-Food': 5, 'I-Interior': 6, 'I-Price': 7,
    'I-Service': 8, 'I-Whole': 9, 'O': 10}

id2label = {idx: label for label, idx in label2id.items()}

In [19]:
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels, label2id):
        self.tokens = tokens
        self.labels = labels
        self.label2id = label2id

    def prepare_data(self, tokens, labels):
        tokens = torch.tensor([tokenizer.convert_tokens_to_ids(text) for text in tokens])
        labels = torch.tensor([self.label2id[l] for l in labels])
        return tokens, labels

    def __getitem__(self, idx):
        tokens, labels = self.prepare_data(self.tokens[idx], self.labels[idx])
        return {'input_ids': tokens, 'labels': labels}

    def __len__(self):
        return len(self.tokens)

In [20]:
train_dataset = NERDataset(train_tokens, train_labels, label2id)
test_dataset = NERDataset(test_tokens, test_labels, label2id)

### Обучение

In [21]:
data_collator = DataCollatorForTokenClassification(tokenizer)

model = BertForTokenClassification.from_pretrained(
    model_name,
    num_labels = len(label2id),
    output_attentions = False,
    output_hidden_states = False
)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
metric = load_metric("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
model_path_name = 'deeppavlov_cased_1e04'
training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/NLP Course/fine_tune_bert_output_{model_path_name}",
    evaluation_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

In [27]:
model_path_name = 'deepvk_uncased_1e02'
model_path_to_save = f'/content/drive/MyDrive/NLP Course/bert_wo_crf_{model_path_name}'
trainer.save_model(model_path_to_save)

### Инференс

In [40]:
classifier = pipeline("ner", model=model_path_to_save)

def get_entities(text_id, text):
    result = classifier(text)
    to_return = []
    words = []
    entities = []
    prev_entity = ''
    positions = []

    for i, token in enumerate(result):
        entity_id = int(token['entity'].split('_')[-1])
        entity = id2label[entity_id]
        start_pos = token['start']
        end_pos = token['end']

        # если токен начинается с ##, то значит, что он является продолжением
        # предыдущего, а значит часть сущности предыдущего токена
        if token['word'].startswith('##'):
            words[-1] += token['word'][2:]
            positions[-1][1] = end_pos

        # если лейбл начинается с ##, то значит, что он является продолжением
        # предыдущего (по дефолту считаем, что лейбл, начинающийся с I является
        # продолжением прыдыдущего)
        elif entity.startswith('I') and entities[-1] != 'O':
            words[-1] += f' {token["word"]}'
            positions[-1][1] = end_pos

        else:
            words.append(token['word'])
            entities.append(entity)
            positions.append([start_pos, end_pos])

        prev_entity = entity

    for word, entity, position in zip(words, entities, positions):
        if entity != 'O':
            to_return.append([str(text_id), entity[2:], word, str(position[0]), str(position[1]), 'neutral'])

    return to_return

In [41]:
res_dict = []
test_ids = test_data.id.unique()
for i, row in tqdm(reviews.iterrows()):
    if row['idx'] in test_ids:
        text_dict = get_entities(row['idx'], row['text'])
        res_dict.extend(text_dict)

284it [00:51,  5.56it/s]


In [42]:
to_save = '\n'.join(['\t'.join(result) for result in res_dict])

with open('/content/drive/MyDrive/NLP Course/dev_pred_aspects.txt', 'w') as f:
    f.write(to_save)

### Анализ результатов

In [ ]:
pred_labels = []
true_labels = []
for i in tqdm(range(len(test_clean_tokens))):
    res = classifier(' '.join(test_clean_tokens.iloc[i]))
    preds = []
    for word in res:
        preds.append(id2label[int(word['entity'].split('_')[-1])])
    pred_labels.append(preds)

for i in tqdm(range(len(test_clean_tokens))):
    true_labels.append(test_labels[i][1:-1])

In [44]:
from seqeval.metrics import classification_report

res_dict = classification_report(true_labels, pred_labels, output_dict=True)

In [45]:
pd.DataFrame(res_dict).T

,precision,recall,f1-score,support
Food,0.702602,0.785863,0.741904,481.0
Interior,0.741117,0.776596,0.758442,188.0
Price,0.585366,0.774194,0.666667,31.0
Service,0.680473,0.751634,0.714286,306.0
Whole,0.770732,0.822917,0.795970,192.0
micro avg,0.709629,0.781302,0.743743,1198.0
macro avg,0.696058,0.782241,0.735454,1198.0
weighted avg,0.710879,0.781302,0.744163,1198.0


В целом, у всех сущностей неплохой результат по f1-score. Хуже всего предсказывается сущность Service и Price. Одна из проблем может состоять в том, что сущности Price достаточно мало и модель плохо её выучивает. Кроме этого встречаются пересечения контекстов в разных сущностях. Например контекст "(не) включить в счёт" встречается как в Service, так и в Price, но так как для Price слово "счёт" встречается чаще, то, вероятно, вместо Service будет предсказываться Price.

In [56]:
# Но вот в счет их ВКЛЮЧИТЬ они не забыли
reviews[reviews.idx==34307]

,idx,text
257,34307,"Недавно пришли компанией, давно не были в этом заведении и лучше бы не приходили!!!! Была пятница, людей в зале под завязку, сели за единственный свободный стол, так столы стоят настолько близко, что выйти из- за него невозможно((( но это не самое страшное...... Очень долго несут заказ либо вобще про тебя забывают, все время приходят и переспрашивают( ну если у тебя плохая память- возьми бумагу и записывай) . Мы брали горячее, салатики, пиво и закуски к пиву, которые так и не дождались, хотя по прошествии часа 1 раз даже напомнили, чего мы делать не должны в принципе. Но вот в счет их ВКЛЮЧИТЬ они не забыли!!!! Салаты раньше были вкуснее. Еду приносят черти как, во- первых без приборов, все время нужно напоминать, во вторых, по времени ужасно,1 из нас уже час как поел, а второму еще даже готовить видимо не начали.Официанта то не дождешься, то еще доесть не успел, уже тарелку забирают. В зале стоит галдеж, музыку вобще не слышно. В это заведение мы раньше ходили с удовольствием, что случилось- непонятно. Больше ни ногой!!!! Цены пивного ресторана, а обслуживание дешевой привокзальной пивнушки((((((("


In [60]:
aspects[(aspects.idx==34307) & (aspects.aspect == 'счет')]

,idx,category,aspect,start,end,mood
4260,34307,Service,счет,584,588,negative


In [57]:
# 2 раз не принесли салат, но в счет включили
reviews[reviews.idx==34710]

,idx,text
192,34710,"Удивляюсь отзывам про хорошее обслуживание. Были 4 раза из них только 1 раз нормально обслужили: 1 раз не дали меню, очень долго несли напитки; 2 раз не принесли салат, но в счет включили; 3 раз вообще эпик фейл - хамское и небрежное отношение официантов... Меню на столе нет при этом официантка подходит и спрашивает: ""Что вы выбрали?"" Из чего? После этого нам дали меню ланча (обычное меню видимо только одно на весь ресторан). После этого официантка удалилась и больше к столу не подходила прождав минут 10, позвали другую. Через 3 мин приняла заказ. Вместо заказанного супа принесла другой, через 2 мин. поменяла. Напитки принесла только после десерта (после просьбы принести напитки!!!!). Ужасно одним словом! Да кухня отличная, кеталонский крем чудо. Но где берут таких официантов??? Честно, неприятное впечатление, думала все изменилось, ан нет=("


In [61]:
aspects[(aspects.idx==34710) & (aspects.aspect == 'счет')]

,idx,category,aspect,start,end,mood
3150,34710,Price,счет,174,178,negative


In [58]:
# счет несли довольно долго
reviews[reviews.idx==31071]

,idx,text
202,31071,"Нормальное заведение, но на мой взгляд, не ""великолепное"". Во-первых, очень шумно: зал небольшой и когда все разом разговаривают (а посетителей было много), получается такой гул, что не слышно, о чем говорят твои спутники, сидящие напротив и сами столы расположены довольно близко друг к другу. Во-вторых, довольно-таки дорого. В-третьих, обслуживание: было очень неприятно топтаться в центре зала в верхней одежде (все официанты бегали по залу, никто не встретил и не показал где гардероб - он оказался шкафом между туалетами:)), официантка приносила сначала блюда, а потом уже приборы, счет несли довольно долго, когда уходили, девушка из гардероба выдала не все куртки и ушла... мелочи, но из них складывается впечатление. Интерьер и месторасположение ресторана понравились. Кухня неплохая, никаких претензий, попробовала равиоли с трюфелями и рикоттой (380 р.) и крем-суп из тыквы с пармезаном (380 р.) (так получилось, что выбрала самые недорогие блюда из меню:)). Вино красное ""домашнее"" - 260 рублей/бокал. Чек на 4 девушек с вином получился около 6000."


In [62]:
aspects[(aspects.idx==31071) & (aspects.aspect == 'счет')]

,idx,category,aspect,start,end,mood
3338,31071,Price,счет,588,592,neutral


Кажется, что всё это должно относиться тогда к категории Service, так как оценивается формат работы сотрудников с счётом. Но в двух случаях из трёх это относится к цене.

Ниже есть пример, в котором текст "белого сухого вина", кажется, должно относиться к еде, но размечено как сервис. Это можно было предположить, если белое сухое вино предлагали как комплимент от ресторана, но в предложении написано "заказали".

In [88]:
reviews[reviews.idx==14539]

,idx,text
259,14539,"27 апреля гуляли с женой по Пушкину и случайно зашли перекусить. Заказали шашлыки из свинины, овощи на гриле, картофель ""Айдахо"" по домашнему морсу и по бокалу французского белого сухого вина. Что понравилось. Очень вкусный, большой шашлык на шампуре с оригинальной подставкой. Когда его вносят- горит. Овощи на гриле и ""Айдахо"" тоже очень хороши. Одним словом кухня отличная. Интерьер ""Ресторанчика"" на меня особого впечатления не произвел. За счет того что стены из стеклопакетов ощущение что находишься в теплице. Но в этом тоже что-то есть. Обслуживание мне понравилось, но было одно но: При заказе официантка предложила сама взять морс по 500 млт, а принесли высокие тонкие стаканы по 200 млт. Я сразу не обратил внимания. А в конце когда сказал об этом официантке она очень смутилась. Думаю- не заостри я на этом внимания, в счете была бы сумма за два по 500 млт, а не за два по 200 млт. Впрочем это только мои догадки. Обслуживали нас девушка и молодой человек. Быстро, вежливо , аккуратно. Ждать заказа долго не пришлось. Убирали использованную посуду быстро. Все доброжелательно и с улыбкой. Счет принесли на 2050 руб. На мой взгляд очень не плохое заведение. Рекомендую."


In [89]:
aspects[(aspects.idx==14539) & (aspects.aspect == 'белого сухого вина')]

,idx,category,aspect,start,end,mood
4290,14539,Service,белого сухого вина,173,191,neutral


In [ ]:
for idx in range(len(true_labels)):
    print(idx)
    for t, p, token in zip(true_labels[idx], pred_labels[idx], test_tokens[idx][1:-1]):
        if 'Interior' in t and 'Interior' not in p:
            print(t, p, token, sep='\t')
    print()

## Обучение BERT CRF

In [18]:
class NERDataset(torch.utils.data.Dataset):
    def __init__(self, tokens, labels, label2id):
        self.tokens = tokens
        self.labels = labels
        self.label2id = label2id

    def prepare_data(self, tokens, labels):
        tokens = torch.tensor([tokenizer.convert_tokens_to_ids(text) for text in tokens])
        labels = torch.tensor([self.label2id[l] for l in labels])
        return tokens, labels

    def __getitem__(self, idx):
        tokens, labels = self.prepare_data(self.tokens[idx], self.labels[idx])
        return {'input_ids': tokens, 'labels': labels}

    def __len__(self):
        return len(self.tokens)

    def collate_function(self, batch: List[Dict[str, torch.Tensor]]) -> Dict[str, torch.Tensor]:
        input_ids = [item["input_ids"].clone().detach() for item in batch]
        labels = [item["labels"].clone().detach() for item in batch]
        attention_mask = [torch.ones(len(item)) for item in input_ids]

        return {
            "input_ids": pad_sequence(input_ids, batch_first=True),
            "labels": pad_sequence(labels, batch_first=True),
            "attention_mask": pad_sequence(attention_mask, batch_first=True),
        }

In [19]:
LOG_INF = 10e5


class BertCrf(nn.Module):
    def __init__(
        self,
        num_labels: int,
        bert_name: str,
        dropout: float = 0.2,
        use_crf: bool = True,
    ):
        super().__init__()
        self.num_labels = num_labels
        self.use_crf = use_crf
        self.cross_entropy = nn.CrossEntropyLoss()

        self.bert = AutoModel.from_pretrained(bert_name)

        self.dropout = nn.Dropout(dropout)
        self.hidden2label = nn.Linear(self.bert.config.hidden_size, num_labels)

        self.start_transitions = nn.Parameter(torch.empty(num_labels))
        self.end_transitions = nn.Parameter(torch.empty(num_labels))
        self.transitions = nn.Parameter(torch.empty(num_labels, num_labels))

        nn.init.uniform_(self.start_transitions, -0.1, 0.1)
        nn.init.uniform_(self.end_transitions, -0.1, 0.1)
        nn.init.uniform_(self.transitions, -0.1, 0.1)

    def _compute_log_denominator(self, features: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        seq_len = features.shape[0]

        log_score_over_all_seq = self.start_transitions + features[0]

        for i in range(1, seq_len):
            next_log_score_over_all_seq = torch.logsumexp(
                log_score_over_all_seq.unsqueeze(2) + self.transitions + features[i].unsqueeze(1),
                dim=1,
            )
            log_score_over_all_seq = torch.where(
                mask[i].unsqueeze(1),
                next_log_score_over_all_seq,
                log_score_over_all_seq,
            )
        log_score_over_all_seq += self.end_transitions
        return torch.logsumexp(log_score_over_all_seq, dim=1)

    def _compute_log_numerator(self, features: torch.Tensor, labels: torch.Tensor, mask: torch.Tensor) -> torch.Tensor:
        seq_len, bs, _ = features.shape

        score_over_seq = self.start_transitions[labels[0]] + features[0, torch.arange(bs), labels[0]]

        for i in range(1, seq_len):
            score_over_seq += (
                self.transitions[labels[i - 1], labels[i]] + features[i, torch.arange(bs), labels[i]]
            ) * mask[i]
        seq_lens = mask.sum(dim=0) - 1
        last_tags = labels[seq_lens.long(), torch.arange(bs)]
        score_over_seq += self.end_transitions[last_tags]
        return score_over_seq

    def get_bert_features(
        self, input_ids: torch.Tensor, attention_mask: torch.Tensor
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        hidden = self.bert(input_ids, attention_mask=attention_mask)["last_hidden_state"]
        hidden = self.dropout(hidden)
        return self.hidden2label(hidden), hidden

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: torch.Tensor,
        labels: torch.Tensor,
    ) -> torch.Tensor:
        features, _ = self.get_bert_features(input_ids=input_ids, attention_mask=attention_mask)
        attention_mask = attention_mask.bool()

        if self.use_crf:
            features = torch.swapaxes(features, 0, 1)
            attention_mask = torch.swapaxes(attention_mask, 0, 1)
            labels = torch.swapaxes(labels, 0, 1)

            log_numerator = self._compute_log_numerator(features=features, labels=labels, mask=attention_mask)
            log_denominator = self._compute_log_denominator(features=features, mask=attention_mask)

            return torch.mean(log_denominator - log_numerator)
        else:
            return self.cross_entropy(
                features.flatten(end_dim=1),
                torch.where(attention_mask.bool(), labels, -100).flatten(end_dim=1),
            )

    def _viterbi_decode(self, features: torch.Tensor, mask: torch.Tensor) -> List[List[int]]:
        seq_len, bs, _ = features.shape

        log_score_over_all_seq = self.start_transitions + features[0]

        backpointers = torch.empty_like(features)

        for i in range(1, seq_len):
            next_log_score_over_all_seq = (
                log_score_over_all_seq.unsqueeze(2) + self.transitions + features[i].unsqueeze(1)
            )

            next_log_score_over_all_seq, indices = next_log_score_over_all_seq.max(dim=1)

            log_score_over_all_seq = torch.where(
                mask[i].unsqueeze(1),
                next_log_score_over_all_seq,
                log_score_over_all_seq,
            )
            backpointers[i] = indices

        backpointers = backpointers[1:].int()

        log_score_over_all_seq += self.end_transitions
        seq_lens = mask.sum(dim=0) - 1

        best_paths = []
        for seq_ind in range(bs):
            best_label_id = torch.argmax(log_score_over_all_seq[seq_ind]).item()
            best_path = [best_label_id]

            for backpointer in reversed(backpointers[: seq_lens[seq_ind]]):
                best_path.append(backpointer[seq_ind][best_path[-1]].item())

            best_path.reverse()
            best_paths.append(best_path)

        return best_paths

    def decode(self, input_ids: torch.Tensor, attention_mask: torch.Tensor) -> List[List[int]]:
        features, _ = self.get_bert_features(input_ids=input_ids, attention_mask=attention_mask)
        attention_mask = attention_mask.bool()

        if self.use_crf:
            features = torch.swapaxes(features, 0, 1)
            mask = torch.swapaxes(attention_mask, 0, 1)
            return self._viterbi_decode(features=features, mask=mask)
        else:
            labels = torch.argmax(features, dim=2)
            predictions = []
            for i in range(len(labels)):
                predictions.append(labels[i][attention_mask[i]].tolist())
            return predictions

    def save_to(self, path: str):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        torch.save(self.state_dict(), path)

    def load_from(self, path: str):
        self.load_state_dict(torch.load(path))

In [20]:
def dict_to_device(
    dict: Dict[str, torch.Tensor],
    device: str = "cuda" if torch.cuda.is_available() else "cpu",
):
    for key, value in dict.items():
        dict[key] = value.to(device)
    return dict

def train_ner(
    num_labels: int,
    bert_name: str,
    train_tokens: list,
    train_labels: list,
    test_tokens: list,
    test_labels: list,
    dropout: float,
    batch_size: int,
    epochs: int,
    log_every: int,
    lr_bert: float,
    lr_new_layers: float,
    use_crf: bool = True,
    save_to = None,
    device="cuda" if torch.cuda.is_available() else "cpu",
):
    model = BertCrf(num_labels, bert_name, dropout=dropout, use_crf=use_crf)
    model = model.to(device)
    model.train()

    train_dataset = NERDataset(train_tokens, train_labels, label2id)
    train_data_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=train_dataset.collate_function,
    )

    test_dataset = NERDataset(test_tokens, test_labels, label2id)
    test_data_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        collate_fn=test_dataset.collate_function,
    )

    optimizer = torch.optim.AdamW(
        [
            {"params": model.start_transitions},
            {"params": model.end_transitions},
            {"params": model.hidden2label.parameters()},
            {"params": model.transitions},
            {"params": model.bert.parameters(), "lr": lr_bert},
        ],
        lr=lr_new_layers,
    )

    loss_history = []
    f1 = []

    step = 0
    for epoch in range(1, epochs + 1):
        for batch in tqdm(train_data_loader):
            step += 1

            optimizer.zero_grad()

            batch = dict_to_device(batch, device)

            loss = model(**batch)

            loss.backward()
            optimizer.step()

            loss_history.append(loss.item())

            if step % log_every == 0:
                model.eval()
                predictions = []
                ground_truth = []
                with torch.no_grad():
                    for batch in test_data_loader:
                        labels = batch["labels"].to(device)
                        del batch["labels"]
                        batch = dict_to_device(batch)

                        prediction = model.decode(**batch)

                        for pred, label, mask in zip(prediction, labels, batch["attention_mask"]):
                            label_ = label[mask.bool().cpu()]
                            flatten_prediction = [id2label[l] for l in pred]
                            flatten_labels = [id2label[l.item()] for l in label_]

                            predictions.append(flatten_prediction)
                            ground_truth.append(flatten_labels)

                f1_micro = f1_score(ground_truth, predictions)
                f1.append(f1_micro)
                print(f"f1-score {f1_micro}")
                model.train()

        print(f"Epoch {epoch}/{epochs}")
    if save_to is not None:
        torch.save(model.state_dict(), save_to)

In [ ]:
num_labels = len(label2id)
model_name = 'deepvk/bert-base-uncased' # 'DeepPavlov/rubert-base-cased'
dropout = 0.1
batch_size = 2
epochs = 8
log_every = 107
lr_bert = 1e-05
lr_new_layers = 1e-02
save_to = '/content/drive/MyDrive/NLP Course/bert_crf_deepavlov/bert_crf_deepvk_1e03_20.pth'

train_ner(
    num_labels, model_name, train_tokens,
    train_labels, test_tokens, test_labels, dropout,
    batch_size, epochs, log_every, lr_bert,
    lr_new_layers, save_to=save_to
)

In [22]:
 def get_entities_crf(text_id, text):
    model.eval()
    tokens = tokenizer(text, return_offsets_mapping=True)
    prediction = model.decode(torch.tensor([tokens['input_ids']]), torch.tensor([tokens['attention_mask']]))

    flatten_prediction = [item for sublist in prediction for item in sublist]
    flatten_prediction = [id2label[label] for label in flatten_prediction]

    tokens_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'])

    to_return = []
    words = []
    entities = []
    positions = []

    for token, label, offsets in zip(tokens_text, flatten_prediction, tokens['offset_mapping']):
        start_pos, end_pos = offsets[0], offsets[1]

        if token.startswith('##'):
            words[-1] += token[2:]
            positions[-1][1] = end_pos

        elif label.startswith('I'):
            words[-1] += f' {token}'
            positions[-1][1] = end_pos

        else:
            # if entity != 'O':
            words.append(token)
            entities.append(label)
            positions.append([start_pos, end_pos])

    for word, entity, position in zip(words, entities, positions):
        if entity != 'O':
            to_return.append([str(text_id), entity[2:], word, str(position[0]), str(position[1]), 'neutral'])

    return to_return

In [25]:
num_labels = len(label2id)
dropout = 0.1
model = BertCrf(num_labels, model_name, dropout=dropout, use_crf=True)
model.load_state_dict(torch.load(save_to, map_location=torch.device('cpu') ))

res_dict = []
test_ids = test_clean_labels.index
for i, row in tqdm(reviews.iterrows()):
    if row['idx'] in test_ids:
        text_dict = get_entities_crf(row['idx'], row['text'].lower())
        res_dict.extend(text_dict)

to_save = '\n'.join(['\t'.join(result) for result in res_dict])
with open('/content/drive/MyDrive/NLP Course/bert_crf_deepavlov/dev_pred_aspects_1e03.txt', 'w') as f:
    f.write(to_save)

284it [01:00,  4.70it/s]


In [44]:
def get_preds(text_id, text):
    tokens = tokenizer(text)
    model.eval()
    prediction = model.decode(torch.tensor([tokens['input_ids']]), torch.tensor([tokens['attention_mask']]))
    flatten_prediction = [id2label[label] for label in prediction[0]]

    return flatten_prediction

preds = []
for i, row in tqdm(reviews.iterrows()):
    if row['idx'] in test_clean_tokens.index:
        pred = get_preds(row['idx'], row['text'])
        preds.append(pred)

284it [01:13,  3.84it/s]


In [ ]:
from seqeval.metrics import classification_report

res_dict = classification_report(preds, true_labels, output_dict=True)